In [1]:
import os
import sys
import gzip
import code
import urllib
import importlib
import tensorflow.python.platform
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tqdm
from PIL import Image

In [2]:
COLAB = True
BRANCH = 'tuantu'

In [3]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/Shareddrives/ML_Road_Segmentation/CS-433-project-tuantu/project_road_segmentation'
    os.chdir(drive_path)
    from helpers.colab import mount_and_pull
    BRANCH_NAME = BRANCH
    # mount_and_pull(BRANCH_NAME, drive, os)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%load_ext autoreload
%autoreload 2

from helpers.image_processing import *
from helpers.file_manipulation import *
from helpers.constants import *
from helpers.prediction_checking import *
from helpers.machine_learning import *
from helpers.loss_functions import *
from helpers.submission import *

In [5]:
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
SAVE_MODEL = True
GENERATE_PREDICTION = False  # If True, will generate a CSV to submit on AICrowd

MODEL_NAME = 'unet_2'  # For now, cnn, unet-1, unet-2
SAVE_DIR = MODELS_SAVE_DIR + MODEL_NAME + '/' 
NUM_EPOCHS = 250

In [6]:
mod = importlib.import_module('models.' + MODEL_NAME)
model_function = getattr(mod, MODEL_NAME)

In [7]:
images, groundtruths = [], []
train_idx, test_idx = get_train_test(images, groundtruths, data_augmentation=True, transformations=['hard_mix', 'rotation', 'mix', 'flip'])

Loading data/generated/flip/groundtruth/: 100%|██████████| 100/100 [00:00<00:00, 202.69it/s]


Training features shape :  (900, 400, 400, 3)
Training labels shape :  (900, 400, 400)


In [8]:
X_train = []
y_train = []
for i in train_idx:
    X_train.append(images[i])#[None,:,:,:])
    y_train.append(groundtruths[i])#[None,:,:])
    images[i] = 0
    groundtruths[i] = 0

In [9]:
X_train = np.array(X_train)

In [10]:
y_train = np.array(y_train)

In [14]:
metrics=[tf.keras.metrics.Precision(),
         tf.keras.metrics.Recall(),
         dice_coef
        ]
callbacks = [
             #tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5)
            ]
if RESTORE_MODEL:
    model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'focal_tversky_loss': focal_tversky_loss, 'dice_coef': dice_coef})
else:  
    split = len(X_train) - int(len(X_train) * VALIDATION_SIZE)
    data_train = tf.data.Dataset.from_tensor_slices((X_train[:split], y_train[:split])).batch(10)
    data_val = tf.data.Dataset.from_tensor_slices((X_train[split:], y_train[split:])).batch(10)
    
    #model = model_function()
    #model.compile(optimizer='adam',loss=focal_tversky_loss, metrics=metrics)
    model.fit(data_train,
                        #train_generator, 
                        epochs = NUM_EPOCHS,
                        #steps_per_epoch=4,
                        #validation_split=VALIDATION_SIZE,
                        validation_data=data_val,
                        callbacks=callbacks)

Epoch 1/250
58/58 [==============================] - 24s 410ms/step - loss: 0.6537 - precision: 0.3683 - recall: 0.7452 - dice_coef: 0.3971 - val_loss: 0.6995 - val_precision: 0.2511 - val_recall: 0.8022 - val_dice_coef: 0.3356
Epoch 2/250
58/58 [==============================] - 21s 365ms/step - loss: 0.5498 - precision: 0.4721 - recall: 0.8101 - dice_coef: 0.5117 - val_loss: 0.6778 - val_precision: 0.2501 - val_recall: 0.9914 - val_dice_coef: 0.3513
Epoch 3/250
58/58 [==============================] - 21s 370ms/step - loss: 0.4908 - precision: 0.5411 - recall: 0.7988 - dice_coef: 0.5776 - val_loss: 0.6630 - val_precision: 0.2708 - val_recall: 0.9713 - val_dice_coef: 0.3696
Epoch 4/250
58/58 [==============================] - 22s 374ms/step - loss: 0.4457 - precision: 0.6025 - recall: 0.7890 - dice_coef: 0.6290 - val_loss: 0.5948 - val_precision: 0.3459 - val_recall: 0.9338 - val_dice_coef: 0.4462
Epoch 5/250
58/58 [==============================] - 22s 378ms/step - loss: 0.4114 - pre

KeyboardInterrupt: ignored

In [15]:
from datetime import datetime
if True:
    now = datetime.now().strftime("%m-%d-%Y_%H-%M-%S") + '/'
    model.save('model_save/unet_2/900_batch_10')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_save/unet_2/900_batch_10/assets


In [ ]:
train_predictions = model.predict(X_train).squeeze()
test_predictions = model.predict(X_test).squeeze()

In [ ]:
visualize_random_predictions(X_train, y_train, train_predictions, size=10)

In [ ]:
visualize_random_predictions(X_test, y_test, test_predictions, size=10)

In [ ]:
data_gen = compute_entire_images_metrics(y_test, test_predictions)

In [ ]:
data_gen

In [ ]:
binary cross entropy 0.852952
dice loss 0.860928

In [17]:
predict_submissions(model)

Loading data/test_set_images/: 100%|██████████| 50/50 [00:19<00:00,  2.61it/s]


Predicting test images...
Creating submission csv at location submissions/submission_12-14-2020_00-34-47.csv
Writing predictions in folder data/predictions/


In [16]:
if COLAB:
    from helpers.colab import download_model
    from google.colab import files
    download_model('900_batch_10', 'model_save/unet_2/900_batch_10', files)

  adding: model_save/unet_2/900_batch_10/ (stored 0%)
  adding: model_save/unet_2/900_batch_10/variables/ (stored 0%)
  adding: model_save/unet_2/900_batch_10/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model_save/unet_2/900_batch_10/variables/variables.index (deflated 79%)
  adding: model_save/unet_2/900_batch_10/assets/ (stored 0%)
  adding: model_save/unet_2/900_batch_10/saved_model.pb (deflated 91%)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>